# Dipole equivalence relations

A molecular system subjected to an external field is described by the Hamiltonian

$$
\hat{H} = \hat{H}_0 + \hat{V}(t)
$$

where field coupling is contained in the latter term

$$
\hat{V}(t) = \sum_\omega \hat{V}^\omega_\alpha F^\omega_\alpha e^{-i\omega t}
$$

With use of a variational approach to study an observable described by the operator $\hat{\Omega}$, the following relation can be found for the associated linear responses {cite}`Norman2018`

$$
-i\omega \langle \langle \hat{\Omega}; \hat{V}^\omega_\alpha \rangle \rangle =
\frac{1}{i\hbar}
\left(
\langle \langle \left[ \hat{\Omega}, \hat{H}_0 \right]; \hat{V}^\omega_\alpha \rangle \rangle_\omega
+
\langle \left[ \hat{\Omega}, \hat{V}^\omega_\alpha \right] \rangle
\right)
$$

In the complete basis set limit, the following commutator relation holds true

$$
\left[
\hat{r}_\alpha, \hat{H}_0
\right] =
\frac{i\hbar}{m_\mathrm{e}} \hat{p}_\alpha
$$

where $m_\mathrm{e}$ is the electron mass.

Combining the two equation for the case when $\hat{\Omega}$ and $\hat{V}^\omega_\alpha$ are both equal to position operators, we are led to conclude

$$
\langle \langle \hat{r}_\alpha; \hat{r}_\beta \rangle \rangle_\omega =
\frac{i}{m_\mathrm{e} \omega} \langle \langle \hat{p}_\alpha; \hat{r}_\beta \rangle \rangle_\omega
$$

A residue analysis results in a similar relation for individual transition moments

$$
\langle \Psi_0 | \hat{r}_\alpha | \Psi_f \rangle =
\frac{i}{m_\mathrm{e} \omega_{f0}} \langle \Psi_0 | \hat{p}_\alpha | \Psi_f \rangle 
$$

where $\hbar\omega_{f0}$ is the transition energy between the ground state $|\Psi_0\rangle$ and the excited state $|\Psi_f\rangle$.

Relations of this form are known as dipole length–dipole velocity equivalences, and they serve the computational chemist with a choice of adopting the length or velocity gauge in calculations of spectra and properties.

As an example, let us calculate the polarizability of the neon atom in the length and velocity representations
and see how results get to be in closer agreement with increasing quality of the employed basis set.

In [1]:
import numpy as np
import veloxchem as vlx

In [2]:
molecule = vlx.Molecule.read_str("Ne  0.000 0.000 0.000")

In [3]:
scf_drv = vlx.ScfRestrictedDriver()
scf_drv.ostream.mute()

lrf_drv = vlx.LinearResponseSolver()
lrf_drv.ostream.mute()

lrf_drv.b_operator = "electric dipole"

lrf_drv.a_components = ["z"]
lrf_drv.b_components = ["z"]

w = 0.0656
lrf_drv.frequencies = [w]

In [18]:
result_string = ""

for basis_set in ["aug-cc-pVDZ", "daug-cc-pVDZ"]:
    basis = vlx.MolecularBasis.read(molecule, basis_set, ostream=None)

    scf_results = scf_drv.compute(molecule, basis)

    lrf_drv.a_operator = "electric dipole"
    lrf_results = lrf_drv.compute(molecule, basis, scf_results)
    lrf_length = lrf_results["response_functions"][("z", "z", w)]

    lrf_drv.a_operator = "linear momentum"
    lrf_results = lrf_drv.compute(molecule, basis, scf_results)
    lrf_velocity = lrf_results["response_functions"][("z", "z", w)]

    result_string += f"{basis_set :>12s}{-lrf_length :16.6f}{lrf_velocity / w :16.6f}\n"

print("-" * 44 + f"\n{'Basis set':12s}{'-<<z;z>>':>16s}{'-i/w*<<p_z;z>>':>16s}\n" + "-" * 44)
print(result_string + "-" * 44)

--------------------------------------------
Basis set           -<<z;z>>  -i/w*<<p_z;z>>
--------------------------------------------
 aug-cc-pVDZ        1.837316        1.720512
daug-cc-pVDZ        2.362190        2.377629
--------------------------------------------


For a property such as the polarizability the addition of diffuse functions in the basis set is critical, and it is seen that with a double augmentation, results obtained in the length and velocity representations are in good mutual agreement.